# A Rudimentary Test for Classical Dimension

Alice receives an input $x\in\{0,1,2,3\}$ and sends it to Bob using 2 classical bits ($|0\rangle$, $|1\rangle$ qubits). Bob measures in the computational basis and outputs the result. The success probability for quantum and classical scenarios is bounded by  $d/N \geq 1/N\sum_x p(b=x,x)$, where $d$ is the number of dimensions in the hilbert space and $N$ is the number of inputs $x$. 

* The first test considers when Alice and Bob verify the ability to send two bits.
* The second test considers when Alice encodes the input onto a single bit. 

In [1]:
from qiskit import QuantumCircuit,execute, IBMQ
from qiskit.tools.monitor import *
from qiskit.providers.ibmq.managed import IBMQJobManager

provider = IBMQ.load_account()

import matplotlib.pyplot as plt
import numpy as np

import context
from device_independent_test import dimension
from device_independent_test.handshake import HandShake
from device_independent_test.quantum_communicator import LocalDispatcher

In [2]:
bit_circ = dimension.prepare_bit_circuit([0,1,1,0,1])

bit_circ.draw()

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     └───┘
q_3: ─────
     ┌───┐
q_4: ┤ X ├
     └───┘

## Test 1)

Alice and Bob can prepare and measure a set of orthogonal measurements on a 4-dimensional Hilbert space. Alice simply encodes the input $x\in\{0,1,2,3\}$ into the bit string and Bob decodes it without error. Assuming a uniform prior distribution for $x$, the success probability for this case is $1 \geq 1/4 \sum_x p(b=x|x)$.

In [3]:
# alice prepares the complete set of orthogonal states

qc_00 = dimension.prepare_bit_circuit([0,0])
qc_01 = dimension.prepare_bit_circuit([0,1])
qc_10 = dimension.prepare_bit_circuit([1,0])
qc_11 = dimension.prepare_bit_circuit([1,1])

circuits = [qc_00, qc_01, qc_10, qc_11]

# Bob measures in the computational basis.
for qc in circuits:
    qc.measure_all()
    
for qc in circuits:
    display(qc.draw())


░ ┌─┐   
   q_0: ─░─┤M├───
         ░ └╥┘┌─┐
   q_1: ─░──╫─┤M├
         ░  ║ └╥┘
meas_0: ════╩══╬═
               ║ 
meas_1: ═══════╩═

░ ┌─┐   
   q_0: ──────░─┤M├───
        ┌───┐ ░ └╥┘┌─┐
   q_1: ┤ X ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas_0: ═════════╩══╬═
                    ║ 
meas_1: ════════════╩═

┌───┐ ░ ┌─┐   
   q_0: ┤ X ├─░─┤M├───
        └───┘ ░ └╥┘┌─┐
   q_1: ──────░──╫─┤M├
              ░  ║ └╥┘
meas_0: ═════════╩══╬═
                    ║ 
meas_1: ════════════╩═

┌───┐ ░ ┌─┐   
   q_0: ┤ X ├─░─┤M├───
        ├───┤ ░ └╥┘┌─┐
   q_1: ┤ X ├─░──╫─┤M├
        └───┘ ░  ║ └╥┘
meas_0: ═════════╩══╬═
                    ║ 
meas_1: ════════════╩═

In [4]:
# running tests on quantum computer

def run_job(qc, shots):
    job = execute(qc, backend=provider.get_backend('ibmq_qasm_simulator'), shots=shots)
    job_monitor(job)
    return job

shots = 1000
d4_job_00 = run_job(qc_00, shots)
d4_job_01 = run_job(qc_01, shots)
d4_job_10 = run_job(qc_10, shots)
d4_job_11 = run_job(qc_11, shots)

Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run


In [5]:
# parsing statistics
d4_counts_00 = d4_job_00.result().get_counts()
d4_counts_01 = d4_job_01.result().get_counts()
d4_counts_10 = d4_job_10.result().get_counts()
d4_counts_11 = d4_job_11.result().get_counts()

# success probability for a d=4 system is 1
p_succ_00 = d4_counts_00["00"]/shots
p_succ_01 = d4_counts_01["10"]/shots # qiskits labeling is weird
p_succ_10 = d4_counts_10["01"]/shots # qiskits labeling is weird
p_succ_11 = d4_counts_11["11"]/shots

d4_success_probability = (p_succ_00 + p_succ_01 + p_succ_10 + p_succ_11)/4 # divide by 4 because there are 4 inputs.

d4_success_probability

1.0

Above test can be run as the following with specific tolerance and shots.

In [2]:

dispatcher = LocalDispatcher([provider.get_backend('ibmq_qasm_simulator')])
handshake = HandShake(dispatcher)

tolerance = 0.1
shots = 1000
handshake.dimensionality(tolerance, shots)


Passed Dimensionality with value:  1.0


(True, 1.0)

In [3]:

dispatcher = LocalDispatcher([provider.get_backend('ibmq_london')])
handshake = HandShake(dispatcher)

tolerance = 0.1
shots = 1000
handshake.dimensionality(tolerance, shots)


Passed Dimensionality with value:  0.9065


(True, 0.9065)

## Test 2)

Alice can only encode information in one classical bit, the second bit is constant. The measurement success_probability is $0.5 \geq 1/4 \sum_x p(b=x|x)$.

In [ ]:
shots = 1000
d2_job_00 = run_job(qc_00, shots)
d2_job_01 = run_job(qc_00, shots)
d2_job_10 = run_job(qc_10, shots)
d2_job_11 = run_job(qc_10, shots)

In [ ]:
counts_00 = d2_job_00.result().get_counts()
counts_01 = d2_job_01.result().get_counts()
counts_10 = d2_job_10.result().get_counts()
counts_11 = d2_job_11.result().get_counts()

d2_p_succ_00 = counts_00["00"]/shots if ("00" in counts_00) else 0.0
d2_p_succ_01 = counts_01["10"]/shots if ("10" in counts_01) else 0.0 # qiskits labeling is weird
d2_p_succ_10 = counts_10["01"]/shots if "01" in counts_10 else 0.0 # qiskits labeling is weird
d2_p_succ_11 = counts_11["11"]/shots if "11" in counts_11 else 0.0

d2_success_probability = (d2_p_succ_00 + d2_p_succ_01 + d2_p_succ_10 + d2_p_succ_11)/2

d2_success_probability # the classical bound is 0.5 